# parameter scan
## env set-up
Julia-lang 1.3.1: `PyCall` (1.91.2), `DataFrames` (0.20.0), `SymPy` (1.0.18)

In [4]:
using PyCall, DataFrames, SymPy
sympy.init_printing() ## use other py3 function not implemented in native Julia-lang, http://mth229.github.io/symbolic.html
println()

## parameters

In [5]:
@vars C P B
T_k,
E_PV,g_P0,r_P0,m_P0,a_P0,e_P,
E_BV,g_B0,r_B0,m_B0,e_B =
symbols(
    "T_k 
E_P g_P0 r_P0 m_P0 a_P0 e_P 
E_B g_B0 r_B0 m_B0 e_B",
    positive = true)
R=8.31

8.31

In [6]:
E_P = E_PV*1.6e-19 # unit conversion from eV to J
E_B = E_BV*1.6e-19

g_P = g_P0*exp(-E_P/(R*T_k))
r_P = r_P0*exp(-E_P/(R*T_k))
m_P = m_P0*exp(-E_P/(R*T_k))
a_P = a_P0*exp(-E_P/(R*T_k))

g_B = g_B0*exp(-E_B/(R*T_k))
r_B = r_B0*exp(-E_B/(R*T_k))
m_B = m_B0*exp(-E_B/(R*T_k))

C_eq = (g_P*(1 -e_P) +a_P*P +m_P)*P + (g_B*(1 -e_B)*C +m_B)*B
P_eq = (g_P*e_P -a_P*P -r_P -m_P)*P
B_eq = (g_B*e_B*C -r_B -m_B)*B

  ⎛            -0.120336943441637⋅E_B          -0.120336943441637⋅E_B         
  ⎜            ───────────────────────         ───────────────────────        
  ⎜                       Tₖ                              Tₖ                  
B⋅⎝C⋅e_B⋅g_B0⋅ℯ                        - m_B0⋅ℯ                        - r_B0⋅

 -0.120336943441637⋅E_B ⎞
 ───────────────────────⎟
            Tₖ          ⎟
ℯ                       ⎠

## default parameter values

In [ ]:
E_PV = .57 # https://www.nature.com/articles/ngeo2807/ FROM [https://link.springer.com/article/10.1007/s11120-014-0067-8 ; https://nph.onlinelibrary.wiley.com/doi/abs/10.1111/j.1469-8137.1988.tb00282.x]
